Referred 
https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb#scrollTo=ovpZyIhNIgoq

In [176]:
import tensorflow as tf
import numpy as np
import os
import time

#Added by Tomb 
import pathlib

In [177]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

#Added by Tomb
path_to_file = pathlib.Path('/Volumes/MAGIC1/CS50/myMusicGen/data/shakespeare.txt')
if not path_to_file.exists():
  tf.keras.utils.get_file(
      'shakespeare.txt', # your desired file name, you can even name it tomato.mid but the convention is to get the name from the end of the URL path
      origin='https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

# path_to_file = pathlib.Path('/Volumes/MAGIC1/CS50/myMusicGen/data/MJLyrics.txt')

In [178]:
# Read then decode for py2 compat (python 2 compatibility)
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [179]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [180]:
# the unique characters in the file, meaning each character that is used 
vocab = sorted(set(text)) # return as a list
print ([text for text in enumerate(vocab)])
print(f'{len(vocab)} Unique characters')

[(0, '\n'), (1, ' '), (2, '!'), (3, '$'), (4, '&'), (5, "'"), (6, ','), (7, '-'), (8, '.'), (9, '3'), (10, ':'), (11, ';'), (12, '?'), (13, 'A'), (14, 'B'), (15, 'C'), (16, 'D'), (17, 'E'), (18, 'F'), (19, 'G'), (20, 'H'), (21, 'I'), (22, 'J'), (23, 'K'), (24, 'L'), (25, 'M'), (26, 'N'), (27, 'O'), (28, 'P'), (29, 'Q'), (30, 'R'), (31, 'S'), (32, 'T'), (33, 'U'), (34, 'V'), (35, 'W'), (36, 'X'), (37, 'Y'), (38, 'Z'), (39, 'a'), (40, 'b'), (41, 'c'), (42, 'd'), (43, 'e'), (44, 'f'), (45, 'g'), (46, 'h'), (47, 'i'), (48, 'j'), (49, 'k'), (50, 'l'), (51, 'm'), (52, 'n'), (53, 'o'), (54, 'p'), (55, 'q'), (56, 'r'), (57, 's'), (58, 't'), (59, 'u'), (60, 'v'), (61, 'w'), (62, 'x'), (63, 'y'), (64, 'z')]
65 Unique characters


## Vectorize the text

Before training, you need to convert the strings to a numerical representation.<br>

In [181]:
# It needs the text to be split into tokens first.
# the 'b' prefix printed out indicates that these characters are stored as bytes objects.
# When text data is read or processed in Python, especially with encoding methods like UTF-8, each character is represented as a byte in memory

example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8') # contains a tensor where each element corresponds to a token (character) from the input texts provided in the example_texts list.
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [182]:
# Now create the tf.keras.layers.StringLookup layer:
# This is NOT zero index. It starts from 1. and the IDs given are unique indices from the vocab variable. "a"'s index is 39 in vaocab as zero index
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary = list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [183]:
# Invert back the vectors of IDs to Chars
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [184]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1) # chars_from_ids is an instance obj of tf.keras.layers.StringLookup, Joins all strings into a single string, or joins along an axis.

# To make sure if it works
# x = text_from_ids(ids)
# print("x!!", x)

## Create Training Examples and Targets

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

In [185]:
"""First prepare IDs, then use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices."""
# tf.strings.unicode_split makes each chars in text tokens
# ids_from_chars variable has a class of tf.keras.layers.StringLookup in it which converts tokens into ids
# print("tokens!!>>>", tf.strings.unicode_split(text, 'UTF-8'))
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
print("IDs!!>>>",all_ids)

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) # ids_dataset becomes a class
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

seq_length = 100

IDs!!>>> tf.Tensor([19 48 57 ... 46  9  1], shape=(1115394,), dtype=int64)
F
i
r
s
t
 
C
i
t
i


In [186]:
"""Convert these individual characters to sequences of the desired size"""
# The batch method lets you easily do that.
#  seq_length + 1, the model learns to predict the next character in the sequence given a fixed window of characters as input.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))


tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [187]:
# It's easier to see what this is doing if you join the tokens back into strings:
for seq in sequences.take(1):
    print(text_from_ids(seq).numpy()) # text_from_ids is the custom function

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#### For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. <br> At each time step the input is the current character and the label is the next character.

In [188]:
# Details below, but this function slides one char to right
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target) #  map automatically takes an argument. dataset is an instance obj of the _MapDataset class

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches
You used **tf.data** to split the text into manageable sequences. <br>
But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [189]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE) # This does Asynchronous data loading, i.e. improves the loading speed
)
# dataset
print(list(dataset.take(1).as_numpy_iterator()))

[(array([[47,  6, 43, ..., 64, 54, 60],
       [26, 64,  2, ..., 58,  2, 58],
       [51, 45,  2, ..., 40,  2, 51],
       ...,
       [32,  2, 37, ..., 53,  2, 40],
       [60, 58, 48, ..., 45, 54, 57],
       [45,  2, 14, ..., 41, 58, 44]]), array([[ 6, 43,  2, ..., 54, 60,  2],
       [64,  2, 55, ...,  2, 58, 47],
       [45,  2, 59, ...,  2, 51, 48],
       ...,
       [ 2, 37, 22, ...,  2, 40, 53],
       [58, 48, 53, ..., 54, 57, 43],
       [ 2, 14, 53, ..., 58, 44, 53]]))]


## Build the Model

In [190]:
# Length of the vocabulary in StringLookup layer
vocab_size =len(ids_from_chars.get_vocabulary())
print([x for x in enumerate(ids_from_chars.get_vocabulary())])
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

[(0, '[UNK]'), (1, '\n'), (2, ' '), (3, '!'), (4, '$'), (5, '&'), (6, "'"), (7, ','), (8, '-'), (9, '.'), (10, '3'), (11, ':'), (12, ';'), (13, '?'), (14, 'A'), (15, 'B'), (16, 'C'), (17, 'D'), (18, 'E'), (19, 'F'), (20, 'G'), (21, 'H'), (22, 'I'), (23, 'J'), (24, 'K'), (25, 'L'), (26, 'M'), (27, 'N'), (28, 'O'), (29, 'P'), (30, 'Q'), (31, 'R'), (32, 'S'), (33, 'T'), (34, 'U'), (35, 'V'), (36, 'W'), (37, 'X'), (38, 'Y'), (39, 'Z'), (40, 'a'), (41, 'b'), (42, 'c'), (43, 'd'), (44, 'e'), (45, 'f'), (46, 'g'), (47, 'h'), (48, 'i'), (49, 'j'), (50, 'k'), (51, 'l'), (52, 'm'), (53, 'n'), (54, 'o'), (55, 'p'), (56, 'q'), (57, 'r'), (58, 's'), (59, 't'), (60, 'u'), (61, 'v'), (62, 'w'), (63, 'x'), (64, 'y'), (65, 'z')]



This model has 3 layers <br>
-tf.keras.layers.Embedding: The input layer. <br>
-tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an **LSTM** layer here.) <br>
-tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. <br>

Make those as instance variables

In [191]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
    
    # The call method defines the forward pass of the model.
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training) 
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x
        

In [192]:
# Make an instant object
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

## Try the Model
Run the model to see that it behaves as expected.<br>
First check the shape of the output:

In [193]:
for input_example_batch, target_example_batch in dataset.take(1): # the len of input and target is both 64
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, seq_length, vocab_size)")
    
    # the UNK token is added on vocab_size =len(ids_from_chars.get_vocabulary()) if you use Lookup layer, the size will automatically differ fron the original vocab (65) to vocab_size (66) 
model.summary()

(64, 100, 66) # (batch_size, seq_length, vocab_size)
Model: "my_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  16896     
                                                                 
 gru_4 (GRU)                 multiple                  3938304   
                                                                 
 dense_4 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [194]:
"""To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. 
This distribution is defined by the logits over the character vocabulary.
Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop."""

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) # 2D tensor
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy() # 1D tensor
print(sampled_indices)

[ 1 21 33 28 11 53 38 51 50 38  0 31 27 55 40  6 52 51  3 54 65  3 57 36
 53  4 28 42 18 40  8 60 62 45 39 43 33 36 52  4 58 27 65 18 38  2 50 62
 65 11  3 50 32 28 26 12 19 22 37  2 14 16 24 53  1 12  1 12 45 64  7  9
 51 53  1  4 24 54 24 43 12 52 18 17 23 15 35 24 21 47  7 58  3 37 59 37
 39 15 55 24]


In [195]:
"""Decode these to see the text predicted by this untrained model:"""
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'own, his forehead, nay, the valley,\nThe pretty dimples of his chin and cheek,\nHis smiles,\nThe very m'

Next Char Predictions:
 b"\nHTO:nYlkY[UNK]RNpa'ml!oz!rWn$OcEa-uwfZdTWm$sNzEY kwz:!kSOM;FIX ACKn\n;\n;fy,.ln\n$KoKd;mEDJBVKHh,s!XtXZBpK"


In [196]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
# print("y_true", target_example_batch)
# print("y_pred", example_batch_predictions)
# print("Prediction shape", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Mean loss:         tf.Tensor(4.1887255, shape=(), dtype=float32)


In [197]:
"""A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. 
To confirm this you can check that the exponential of the mean loss is approximately EQUAL to the vocabulary size. 
(A much higher loss means the model is sure of its wrong answers, and is badly initialized:)"""
tf.exp(example_batch_mean_loss).numpy() # Exponential e = 2.718.. power to 4.189.. = 65.99708..

65.9387

In [198]:
model.compile(optimizer="adam", loss=loss)

### Configure checkpoints
Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [199]:
# Directort where the checkpoints will be save
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

### Execute the training

In [200]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback]) 

Epoch 1/3


172/172 [==============================] - 152s 872ms/step - loss: 2.7289
Epoch 2/3
172/172 [==============================] - 145s 836ms/step - loss: 1.9906
Epoch 3/3
172/172 [==============================] - 147s 848ms/step - loss: 1.7117


In [213]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, 
                                          states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [214]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [215]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun Time:', end - start)

ROMEO:
'Tis to thy Excarse.
You how, but mastleman: ferved. And that with a'll,
And then furm the kind
And laddias of my gigethand badcis frown it
Trwat hath burd the queen, is thee.
Graye, depy it.

GRETIO:
Pray, for thize, sir, us Iw's Claitors, is thou,
'Tiss for may what vains langer.

CORIOLANUS:
Be gord, he wark 'dill have doo:
That will dutruchio's away to the deweren, prince, Pory.

KATHARGIA:
Nor oft with one pacess first as befter.

QUEEN SARKE:
No, not in to whem
And mant thou dveet surst not hack that tomene me not pally is
Beacke than thy royo! makt that your like!

MENENCE:
Whack not whyse wear'd fit
with your worndry queen of you taulte my peace, 'scopes in thou when,
Have mead spring abfuction be
namer. Take your whore copes it ally
Dose friends is shall bain, Wit, a forloway, efe, then, sood says as she nevies
O' the curpaty,
For not but that say on you. Thy house fould seed o' the pleecke what his lawy,
And though meas you go; I faul gevat awst;
'CComio! Ladt wain,
Sh

In [ ]:
"""Save the model"""

In [ ]:
"""About tf.keras.layersStringLookup()"""

# vocab1 = ["a", "b", "c", "d"]
# data = tf.constant([["a", "c", "d"],["d", "z", "b"]])
# print(data)
# layer = tf.keras.layers.StringLookup(vocabulary=vocab1)
# layer(data)

# vocab = ['apple', 'banana', 'orange']
# string_lookup = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token='UNK') # UNK refers unknown and it gives the unknown element the index 0. StringLookup starts at 1 index

# ids = string_lookup(['apple', 'kiwi', 'orange'])
# print(ids.numpy())

#### ---Unused or test code below---

In [205]:
"""about batch in tf.data.Dataset"""
# dataset = tf.data.Dataset.range(8)
# dataset = dataset.batch(3)
# print(list(dataset.as_numpy_iterator()))


# dataset1 = tf.data.Dataset.range(8)
# dataset1 = dataset1.batch(3, drop_remainder=True)
# print(list(dataset1.as_numpy_iterator()))

'about batch in tf.data.Dataset'

In [206]:
"""About take() in tf.data.Dataset"""
# dataset = tf.data.Dataset.range(10)
# dataset = dataset.take(1)
# list(dataset.as_numpy_iterator())

# list(sequences.as_numpy_iterator())

# seq = sequences.take(1)
# list(seq.as_numpy_iterator())

'About take() in tf.data.Dataset'

In [207]:
"""How the split_input_target function works"""

# sequence = ["H", "e", "l", "l", "o"]
# sequence = list("Hello") # equivalent above but more convenient

# input, target = split_input_target(sequence)
# print(input)
# print(target)

'How the split_input_target function works'

In [208]:
"""About reduce_join"""
# join = tf.strings.reduce_join([['abc','123'],
#                         ['def','456']]).numpy()

# join1 = tf.strings.reduce_join([['abc','123'],
#                         ['def','456']], axis=-2).numpy() # axis=0 or -2 is column, axis=1 or -1 is row

# join2 = tf.strings.reduce_join([['abc','123'],
#                         ['def','456']],
#                        axis=-1,
#                        separator="<<<Tomb>>>").numpy()

# print(join)
# print(join1)
# print(join2)



'About reduce_join'

In [209]:
"""About tf.keras.losses.SparseCategoricalCrossentropy()
measures the difference between the true labels (y_true) and the predicted probabilities (y_pred). """
# y_true = [1, 2]
# y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# # Using 'auto'/'sum_over_batch_size' reduction type.
# scce = tf.keras.losses.SparseCategoricalCrossentropy()
# scce(y_true, y_pred).numpy()


'About tf.keras.losses.SparseCategoricalCrossentropy()\nmeasures the difference between the true labels (y_true) and the predicted probabilities (y_pred). '

In [210]:
"""About float('inf')"""
# <Set upper bound>
# value = float("inf") # this is the only way to print (value exceeds). Any number can not be greater than float("inf") which is infinite
# max_value = float("inf")
# # Check if a value exceeds the maximum allowed
# if value >= max_value:
#     print("Value exceeds the maximum allowed.")
# else:
#     print("Yes")





"About float('inf')"

In [211]:
"""About @tf.function"""
# official usage: tf.range(start, limit, delta=1, dtype=None, name='range')

# start = 0
# limit = 16
# delta = 1 # this is like "step"
# p = (tf.range(start, limit, delta))
# p = tf.reshape(p, [4,4]).numpy()
# print(p)


# @tf.function  # The decorator converts `add` into a `PolymorphicFunction` internally but not human-eye-visible.
# def add(a, b):
#   return a + b

# add(tf.ones([2, 2]), tf.ones([2, 2]))

'About @tf.function'